In [1]:
import pandas as pd
import numpy as np
import re
import os
import time
# No Warnings for chain #
pd.options.mode.chained_assignment = None

In [2]:
# Path Setting #
read_path = 'C:\\Finance_team\\raw_data'
save_path = 'C:\\Finance_team\\Testing_result\\profit_loss_KR4165N30007.csv'

# Result csv 파일 초기 Column 설정 #
result_file_columns = \
'Date,Today_profit,Today_order_quantity,\
short_quantity,short_remained_quantity,\
long_quantity,long_remained_quantity,\
short_close_quantity_at_last,long_close_quantity_at_last\n'

# Data Frame 파일 초기 Column 설정 #
df_columns = ['시간', '현재가', '체결량', '투자자', '매수1호가수량',
             '매도1호가수량', '투자자매수량', '투자자매도량', '매도or매수']

df_numeric_columns = ['현재가', '체결량', '투자자정보', '매수1호가수량', 
                      '매도1호가수량', '투자자매수량', '투자자매도량', '매도or매수']

# Raw File 저장 Directory를 찾아서, 그 안의 파일명을 호출하여 List로 변환 #
file_list = os.listdir(read_path)
file_CMDT_list = []

# Directory에서 CMDT File 만 Sorting하여 file_CMDT_list에 Insert
for file in file_list:
    if file[0:19] == 'sise2local.ALL_CMDT':
        file_CMDT_list.append(file)

# Result 파일을 생성하여 open.
result_file = open(save_path, 'a', encoding = 'ISO-8859-1')

# 만일, Result file이 처음 만들어 지는 것이라면, result_file_columns을 write하고
# 만일, Result file이 기존에 있었다면, 그냥 pass
if os.path.getsize(save_path) == 0:
    result_file.write(result_file_columns)

result_file.close()

## 변수 초기 설정
product_id = ''
product_code = ''
date = ''

df_columns = ['시간', '현재가', '체결량', '투자자', '매수1호가수량',
             '매도1호가수량', '투자자매수량', '투자자매도량', '매도or매수']

df_numeric_columns = ['현재가', '체결량', '투자자정보', '매수1호가수량', 
                      '매도1호가수량', '투자자매수량', '투자자매도량', '매도or매수']

In [3]:
def is_target_product_ID(line):
    return line[17:19] == 'A0' and line[425:428] == 'BM3' and line[471:474] == '001'

In [ ]:
%%time

# file_CMDT_list 파일을 하나씩 하나씩 읽어서 진행
for i in file_CMDT_list:
    print('Current_Reading_File_Name: ', i)
    with open('{0}\\{1}'.format(read_path, i), 'r', encoding='ISO-8859-1') as f:
        result = []
        lines = f.readlines()
        array_data = []
        
        ## A3: 09:00 이후에 A3가 나오면 A3 = False 에서 A3 = True로 변환
        ## 하여 1번만 Filtering을 하기를 위함.
        A3 = False
        
        ## Target_Product_ID를 찾으면, 그 Data에서 1) product ID 2) product Code 3) date를 변수로 저장. 
        for line in lines:
            if is_target_product_ID(line):
                product_id = line[53:64]
                product_code = line[35:47]
                date = line[27:35]
                print('product_id:', product_id)
                print('product_code:', product_code)
                print('date:', date)
                break

        ## lines에 저장되어 있는 Data를 array로 변환
        ## Why? array로 사용하면 Data 처리 속도가 더 빠름.
        for line in lines:
            array_data.append(np.array(line.split(',')))
        
        
        ## array Data로 실질적으로, 전처리 시작; 
        for line in array_data:
            
            ## 09:00:00-000000 이전 데이터 disregard  
            ## 현재는 필요 없기에 drop하는 것이지만, 추후에 사용 가능.
            if (line[0][1:16] < '09:00:00-000000'): continue # 
            
            ## 09시 이후에 'A3'코드가 나오면, 그 때부터 읽기 시작. 
            ## Why? 장이 열리면 (09:00:00), 동시호가 때 (08:00 ~ 09:00) 주문이 단일 호가로 
            ## 체결되면서, 시작하는게 일반적인데, 가끔씩 A3코드가 나오기 전에 B6코드가 읽힐
            ## 때가 있음. 이렇게 09시 이후에 A3코드가 나오기전에 따른 code가 읽힌다면,
            ## 그 code는 버리고 A3 코드가 나온 이후의 데이터를 읽기 시작함.
            if (line[0][17:19]) == 'A3':
                A3 = True ####### 그런데, 이 코드 가독성 측면에서 비효율적일것 같음  => 알아보자.
                
            
            ## 장 마감 시간 이후에 들어오는 Data 는 버린다.
            ## 이걸 왜 15:45:01로 해놨지? 
            ## 동시호가 (15:35 ~ 15:45 끝난 후, 가격 Data도 보고 싶어서)
            ## 왜 '01'을 붙였나?? --> 다시 확인해봐야 될듯 (이유가 있어서 지난번에 만들었지만)
            ## 설명을 안적어 놔서 잃어 버렸기 때문에, 다시 생각해보자.... 앞으로 뭔짓을 하든 설명적기;;!!
            if (line[0][1:3] == '15' and line[0][4:6] == '45' and line[0][7:9] >= '01'): break
                        
            #########!!!!!!!!!!!!!!!!!!!!!!!!!갑자기 든 생각!!!! 만기 일에는 시간을 다르게 설정해야 함!!!###########
            ###### 이거 따로 구현해야 합니다!!!!!!!!!!!!!!!!!!!!!! 추가하세요!!!!!!!!!!!!!!
            
            ## 이전의 if문에서, 시간을 Filtering 해 놨기 때문에 09:00 ~ 15:45 까지만 Filter 됨
            ## 거기서, 원하는 product_code와 product_id 값을 위에서 설정해 놨기에 
            ## 그 내용이 맞다면, 원하는 코드(A3, B6, G7, H1)만 data frame으로 저장. 
            if ((line[0][22:34] == product_code) or (line[0][38:49] == product_id)) and A3 == True:
                
                ## A3 = 체결 Data, B6 = 우선호가, G7 = 우선호가_체결 
                ## A3, B6, G7 코드에 대한 length별 data의 의미를 함수화 할 수 있음
                if line[0][17:19] == 'A3':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],np.nan,
                                    np.nan, np.nan, np.nan, np.nan, line[0][152:153]])
                elif line[0][17:19] == 'B6':
                    result.append([line[0][:17], np.nan, np.nan, np.nan, line[0][57:63],
                                    line[0][139:145], np.nan, np.nan, np.nan])
                elif line[0][17:19] == 'G7':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],
                                   np.nan, line[0][169:175], line[0][251:257],
                                   np.nan, np.nan, line[0][152:153]])
                elif (line[0][17:19] == 'H1' and (line[0][49:53] == '1000')): ## 1000 금융 기관
                    # H1코드는 주문에만 필요한 signal 이기에 여기서, 15:35분 이후로는 수집하지 않는다. 
                    if (line[0][1:3] == '15' and line[0][4:6] == '35' and line[0][7:9] >= '01'): break
                    result.append([line[0][:17], np.nan, np.nan, line[0][49:53],
                                  np.nan, np.nan, line[0][53:62], line[0][62:71],
                                  np.nan])

        result_frame = pd.DataFrame(result, columns = df_columns)
        result_frame.drop(0, inplace=True)

In [8]:
%%time

# file_CMDT_list 파일을 하나씩 하나씩 읽어서 진행
for i in file_CMDT_list:
    print('Current_Reading_File_Name: ', i)
    with open('{0}\\{1}'.format(read_path, i), 'r', encoding='ISO-8859-1') as f:
        result = []
        lines = f.readlines()
        array_data = []
        
        ## A3: 09:00 이후에 A3가 나오면 A3 = False 에서 A3 = True로 변환
        ## 하여 1번만 Filtering을 하기를 위함.
        A3 = False
        
        ## Target_Product_ID를 찾으면, 그 Data에서 1) product ID 2) product Code 3) date를 변수로 저장. 
        for line in lines:
            if is_target_product_ID(line):
                product_id = line[53:64]
                product_code = line[35:47]
                date = line[27:35]
                print('product_id:', product_id)
                print('product_code:', product_code)
                print('date:', date)
                break

        ## lines에 저장되어 있는 Data를 array로 변환
        ## Why? array로 사용하면 Data 처리 속도가 더 빠름.
        for line in lines:
            array_data.append(np.array(line.split(',')))
        
        
        ## array Data로 실질적으로, 전처리 시작; 
        for line in array_data:
            
            ## 09:00:00-000000 이전 데이터 disregard  
            ## 현재는 필요 없기에 drop하는 것이지만, 추후에 사용 가능.
#             if (line[0][1:16] < '09:00:00-000000'): continue # 
            
            ## 09시 이후에 'A3'코드가 나오면, 그 때부터 읽기 시작. 
            ## Why? 장이 열리면 (09:00:00), 동시호가 때 (08:00 ~ 09:00) 주문이 단일 호가로 
            ## 체결되면서, 시작하는게 일반적인데, 가끔씩 A3코드가 나오기 전에 B6코드가 읽힐
            ## 때가 있음. 이렇게 09시 이후에 A3코드가 나오기전에 따른 code가 읽힌다면,
            ## 그 code는 버리고 A3 코드가 나온 이후의 데이터를 읽기 시작함.
            
            if (line[0][17:19]) == 'A3': ## 이 코드가 09:00:000 없어도 되게 만듦.
                
                A3 = True ####### 그런데, 이 코드 가독성 측면에서 비효율적일것 같음  => 알아보자.
                        
            #########!!!!!!!!!!!!!!!!!!!!!!!!!갑자기 든 생각!!!! 만기 일에는 시간을 다르게 설정해야 함!!!###########
            ###### 이거 따로 구현해야 합니다!!!!!!!!!!!!!!!!!!!!!! 추가하세요!!!!!!!!!!!!!!
            
            ## 이전의 if문에서, 시간을 Filtering 해 놨기 때문에 09:00 ~ 15:45 까지만 Filter 됨
            ## 거기서, 원하는 product_code와 product_id 값을 위에서 설정해 놨기에 
            ## 그 내용이 맞다면, 원하는 코드(A3, B6, G7, H1)만 data frame으로 저장. 
            if ((line[0][22:34] == product_code) or (line[0][38:49] == product_id)) and A3 == True:
                
                ## A3 = 체결 Data, B6 = 우선호가, G7 = 우선호가_체결 
                ## A3, B6, G7 코드에 대한 length별 data의 의미를 함수화 할 수 있음
                if line[0][17:19] == 'A3':
                    if (line[0][1:16] >= '15:35:00-000001'): continue # 현재 동시호가 가격은 필요가 없기 때문에 35분으로 지정.
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],np.nan,
                                    np.nan, np.nan, np.nan, np.nan, line[0][152:153]])
                elif line[0][17:19] == 'B6':
                    if (line[0][1:16] >= '15:35:00-000001'): continue
                    result.append([line[0][:17], np.nan, np.nan, np.nan, line[0][57:63],
                                    line[0][139:145], np.nan, np.nan, np.nan])
                elif line[0][17:19] == 'G7':
                    if (line[0][1:16] >= '15:35:00-000001'): continue
                    
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],
                                   np.nan, line[0][169:175], line[0][251:257],
                                   np.nan, np.nan, line[0][152:153]])
                elif (line[0][17:19] == 'H1') and (line[0][49:53] == '1000'): ## 1000 금융 기관
                    if (line[0][1:16] >= '15:35:00-000001'): continue
                    result.append([line[0][:17], np.nan, np.nan, line[0][49:53],
                                  np.nan, np.nan, line[0][53:62], line[0][62:71],
                                  np.nan])

        result_frame = pd.DataFrame(result, columns = df_columns)
        result_frame.drop(0, inplace=True)

Current_Reading_File_Name:  sise2local.ALL_CMDT.0201
product_id: KRDRVFUBM3 
product_code: KR4165N30007
date: 20180201
Current_Reading_File_Name:  sise2local.ALL_CMDT.0209
product_id: KRDRVFUBM3 
product_code: KR4165N30007
date: 20180209
Wall time: 10.5 s


In [36]:
result_frame.tail()

,시간,현재가,체결량,투자자,매수1호가수량,매도1호가수량,투자자매수량,투자자매도량,매도or매수
11653,[15:34:58-580372],NaN,NaN,NaN,001791,000179,NaN,NaN,NaN
11654,[15:34:58-753005],NaN,NaN,NaN,001791,000179,NaN,NaN,NaN
11655,[15:34:58-823945],NaN,NaN,NaN,001791,000179,NaN,NaN,NaN
11656,[15:34:59-210684],NaN,NaN,NaN,001791,000209,NaN,NaN,NaN
11657,[15:34:59-959178],00010758,000010,NaN,001791,000199,NaN,NaN,2
